In [1]:
from pystac_client import Client
from odc.stac import load
import xarray as xr
import numpy as np
import requests
from dep_tools.grids import PACIFIC_EPSG
from fiona.io import ZipMemoryFile
import odc.geo # noqa
from xarray import DataArray, Dataset
from odc.geo.geom import Geometry
from odc.geo.xr import mask
import geopandas as gpd
import pandas as pd

In [2]:
# site = "Udu"

In [3]:
# client = Client.open("https://stac.digitalearthpacific.org")

catalog = "https://stac.digitalearthpacific.org"
collection = "dep_s2_mangroves"

In [4]:
# # bbox = gpd.read_file("priority_1_sites/"f"{site}.geojson")
# bbox = bbox.to_crs("EPSG:4326")
# bounds_df = bbox.bounds
# # bbox = Geometry(bbox.geometry.values[0], crs=bbox.crs)

In [5]:
# First get the country EEZs
url = "https://files.auspatious.com/unsw/EEZ_land_union_v4_202410.zip"
source_internal_path_name = "EEZ_land_union_v4_202410/EEZ_land_union_v4_202410.shp"
zip_bytes = requests.get(url).content

with ZipMemoryFile(zip_bytes) as z:
    with z.open(source_internal_path_name) as src:
        eez = gpd.GeoDataFrame.from_features(src, crs=src.crs)

In [6]:
# Only do fiji
eez_fiji = eez[eez["ISO_SOV1"] == "FJI"].to_crs("EPSG:4326")

# Convert to ODC Geometry
eez_fiji_geom = Geometry(eez_fiji.geometry.values[0], crs=eez_fiji.crs)

In [7]:
# Get some seagrass data for the area
client = Client.open("https://stac.digitalearthpacific.org")
items = client.search(
    collections=["dep_s2_ammi"],
    intersects=eez_fiji_geom.json
).item_collection()

print(f"Found {len(items)} items")

Found 211 items


In [ ]:
config = {
    collection: {
        "assets": {
            "mangroves": {"data_type": "int16"}
        }
    }
}

data = load(items, bands=["mangroves"], stac_cfg=config)
data

In [ ]:
data = load(
    items,
    bands=["mangroves"],
    crs=PACIFIC_EPSG,  # 
    resolution=100,  # Change to 10 for full resolution,
    chunks={"x": 2048, "y": 2048}
)

# Convert data to 0 and 1, where 

data = data.mangroves != 255

data

In [ ]:
def xarray_calculate_area(
    data: Dataset | DataArray,
    geom: Geometry,
    variable: str | None = None,
    value: int | float | None = None,
) -> float:
    # Work with a dataarray, not a dataset, so it's a singular thing
    if type(data) is not DataArray:
        if variable is None:
            raise ValueError("Variable must be specified when data is a Dataset.")
        data = data[variable]

    # Only select a specific value. This will convert to float, with nans
    if value is not None:
        data = data.where(data == value)

    # Mask out regions outsize the geometry
    masked = mask(data, geom.to_crs(data.odc.crs))

    # Count all the non-nan cells, and multiply by area
    count = float(masked.notnull().sum().values)
    one_pixel_area = abs(
        masked.odc.geobox.resolution.x * masked.odc.geobox.resolution.y
    )

    return float(count) * one_pixel_area

In [ ]:
# Run the xarray_calculate_area per time step
results = []
for time in data.time:
    da = data.sel(time=time)
    area_m2 = xarray_calculate_area(da, eez_fiji_geom, variable="mangroves", value=1)
    # Year, in YYYY, area in m2, area in km2
    results.append({
        "time": pd.to_datetime(time.values).year,
        "area_m2": area_m2,
        "area_ha": area_m2 / 1e4,
        "area_km2": area_m2 / 1e6
    })

df = pd.DataFrame(results)
df

In [ ]:
items

In [ ]:
# Assuming your 'data' DataArray is named 'data'

# Loop through each time step
for time in data.time:
    one_year_data = data.sel(time=time)
    
    for v in values_to_count:
        # 1. Create a boolean mask: True where the value is v
        #    (one_year_data == v) is a Dask-compatible operation
        mask = (one_year_data == v)
        
        # 2. Count the 'True' values (the pixels for value 'v') using .sum()
        #    .sum() is a Dask-compatible reduction. .compute() triggers the calculation.
        pixel_count = mask.sum().compute().item()
        
        # 3. Apply your area conversion (using the corrected 0.01 factor for ha)
        area_ha = pixel_count * 0.01
        
        # 4. Store the result in the count_array
        count_array.loc[{"time": time, "values": v}] = area_ha

In [ ]:
# Define the values you want to count (0, 1, and 2)
values_to_count = [0, 1, 2]

# Initialize an empty DataArray to store the counts
count_array = xr.DataArray(
    np.zeros((len(data["time"]), len(values_to_count))),
    coords={"time": data["time"], "values": values_to_count},
    dims=["time", "values"],
)

# Loop through each value and count occurrences in each year
for time in data.time:
    year = time.values.astype("datetime64[Y]")
    one_year_data = data.sel(time=time)
    count = one_year_data.groupby(one_year_data).count()
    for i, v in enumerate(values_to_count):
        if v not in count:
            # Add the missing count to the array
            count_array.loc[{"time": time, "values": v}] = 0
        else:
            # Add the count to the array
            val = count.sel(mangroves=v)
            count_array.loc[{"time": time, "values": v}] = val * 100 / 10000

# Rename the count variable
count_array = count_array.rename("count")
count_array

In [ ]:
count_array.plot.line(x="time", hue="values")

In [ ]:
count_ha = 0.01

dataframe = count_array.to_dataframe()
results_df = dataframe['count'].unstack(level='time')
x = 4
results_df.columns = results_df.columns.astype(str).str[:x]

dataframe['ha'] = dataframe['count'] * count_ha
dataframe

In [ ]:
# dataframe = count_array.to_dataframe()
# results_df = dataframe['count'].unstack(level='time')
# x = 4
# results_df.columns = results_df.columns.astype(str).str[:x]

In [ ]:
class_mapping = {
    0: 'No mangroves',
    1: 'Low canopy cover',
    2: 'High canopy cover'
}

results_df = results_df.rename(index=class_mapping, level='values')

In [ ]:
results_df = results_df.rename_axis(columns="Hectares over time")
results_df.index.name = None
results_df

In [ ]:
# results_df.plot.line()

In [ ]:
results_df.to_csv("change_detection/"f"{site}-area-cd.csv")

In [ ]:
# for year in range(2018, 2023):
#     data.sel(time=str(year), method='nearest').mangroves.odc.write_cog(f"test_change_detection/mangroves_{year}.tif", overwrite=True)

In [ ]:
# data.to_zarr("test.zarr")

In [ ]:
for time in data.time:
    year = time.values.astype("datetime64[Y]")
    one_year_data = data.sel(time=time)
    # one_year_data.mangroves.odc.write_cog(f"mangroves_{year}.tif")

In [ ]:
mangroves_2018 = data.isel(time=1)
mangroves_2018 = mangroves_2018.astype("uint16")
# mangroves_2018.odc.write_cog("mangroves_ammi_2018.tiff")

In [ ]:
mangroves_2024 = data.isel(time=7)
mangroves_2024 = mangroves_2024.astype("uint16")
# mangroves_2024.odc.write_cog("mangroves_ammi_2024.tiff")

In [ ]:
mangrove_change_2018_2024 = (mangroves_2024*-2) - mangroves_2018

In [ ]:
import numpy as np
# Assuming my_raster is your xarray.DataArray

# 1. Define the old and new No Data values
OLD_NODATA = -1020
NEW_NODATA = -9

# 2. Use the xarray.DataArray.where() method to replace the value.
#    The `where` method keeps the values where the condition is True,
#    and replaces values where the condition is False with the specified value.

change_2018_2024 = change_2018_2024.where(
    change_2018_2024 != OLD_NODATA, # Keep all values that are NOT -1020
    NEW_NODATA               # Replace -1020 with -9
)

# 3. (Optional but recommended) Update the attributes if available
#    This ensures future tools/functions recognize the new No Data value.
if 'nodata' in change_2018_2024.attrs:
    change_2018_2024.attrs['nodata'] = NEW_NODATA
elif '_FillValue' in change_2018_2024.attrs:
    change_2018_2024.attrs['_FillValue'] = NEW_NODATA

print(f"No Data value -1020 has been changed to {NEW_NODATA}")

In [ ]:
from matplotlib import colors

cd_classes = [
    [-9, "No data", "#FFFFFF", "transparent"],
    [-8, "High-density - High-density", "#007c69", "teal"],
    [-7, "High-density - Low-density", "#FFC91C", "orange"],
    [-6, "High-density - No mangroves", "#800600", "maroon"],
    [-5, "Low-density - High-density", "#00801E", "green"],
    [-4, "Low-density - Low-density", "#BBF2C6", "light green"],
    [-3, "Low-density - No-mangroves", "#FFF899", "yellow"],
    [-2, "No-mangroves - High-density", "#600087", "purple"],
    [-1, "No-mangroves - Low-density", "#A97EBD", "lavender"],
    [0, "No-mangroves - No-mangroves", "#E0E0E0", "grey"],
]

values_list = [c[0] for c in cd_classes]
cd_color_list = [c[2] for c in cd_classes]

cd_color_list = [cd[2] for cd in cd_classes]
bounds = values_list + [10]
cd_map = colors.ListedColormap(cd_color_list)
norm = colors.BoundaryNorm(bounds, cd_map.N)

# Do for loop

# Do variable t1 and t2 to replace code below

print("Mangrove extent changes")
change_2018_2024 = data.mangroves.sel(time="2018", method='nearest')*-3 - data.mangroves.sel(time="2024", method='nearest') 



In [ ]:
change_2018_2024

In [ ]:
change_2018_2024.plot(cmap=cd_map, size=5)

In [ ]:
change_2018_2024.odc.write_cog("change_detection/change_2018_2024.tif", overwrite=True)
